In [24]:
%pip install logomaker

     |████████████████████████████████| 11.8MB 13.9MB/s 


In [25]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import numpy as np
import requests as rq
import io, h5py

import pandas as pd
import matplotlib.pyplot as plt
import logomaker
%matplotlib inline

In [2]:
data = rq.get('https://www.dropbox.com/s/c3umbo5y13sqcfp/synthetic_dataset.h5?raw=true')
data.raise_for_status()

with h5py.File(io.BytesIO(data.content), 'r') as dataset:
    x_train = np.array(dataset['X_train']).astype(np.float32).transpose([0, 2, 1])
    y_train = np.array(dataset['Y_train']).astype(np.float32)
    x_valid = np.array(dataset['X_valid']).astype(np.float32).transpose([0, 2, 1])
    y_valid = np.array(dataset['Y_valid']).astype(np.int32)
    x_test = np.array(dataset['X_test']).astype(np.float32).transpose([0, 2, 1])
    y_test = np.array(dataset['Y_test']).astype(np.int32)

print(x_train.shape, y_train.shape)

(21000, 200, 4) (21000, 12)


In [33]:
def generate_model(params):
  filters, kernel, BN1, pool, stride, heads, key_size, head_size, dense, BN2 = params
  inputs = layers.Input(shape=(200, 4))

  nn = layers.Conv1D(filters=filters, kernel_size=kernel, padding='same', use_bias='false')(inputs)
  if BN1:
    nn = layers.BatchNormalization()(nn)
  nn = layers.Activation('relu')(nn)
  if pool != 0:
    nn = layers.MaxPool1D(pool_size=pool, strides=stride)(nn)
  nn = layers.Dropout(0.05)(nn)

  positions = tf.range(nn.shape[1])
  context = layers.Embedding(input_dim=nn.shape[1], output_dim=nn.shape[2])(positions)

  contextual_meaning = tf.add(nn, context)

  query = layers.Dense(key_size, name='query')(contextual_meaning)
  value = layers.Dense(key_size)(contextual_meaning)

  attention, weights = layers.MultiHeadAttention(num_heads=heads, key_dim=head_size, dropout=0.55)(query, value, return_attention_scores=True)

  nn = layers.Flatten()(attention)
  nn = layers.Dense(dense, use_bias=False)(nn)
  if BN2:
    nn = layers.BatchNormalization()(nn)
  nn = layers.Activation('relu')(nn)
  nn = layers.Dropout(0.5)(nn)

  output = layers.Dense(12, activation='sigmoid')(nn)

  model = Model(inputs=inputs, outputs=output)
  model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(curve='ROC', name='auroc'), tf.keras.metrics.AUC(curve='PR', name='aupr')])
  return model

In [35]:
base = [64, 19, True, 4, 4, 8, 100, 32, 512, True]

filters = [4, 8, 16, 32, 64, 96, 128]
kernels = [2, 4, 6, 8, 10, 15, 19]
BN1 = [True, False]
pools = [1, 2, 4, 6, 10, 20, 25, 50, 100]
strides= [1, 2, 4, 6, 10]
heads = [1, 2, 4, 8, 12, 20]
key_sizes = [10, 20, 50, 100, 200]
head_sizes = [4, 8, 16, 32, 64, 96]
denses = [32, 64, 128, 256, 512]
BN2 = [True, False]

model = generate_model(base)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='min', restore_best_weights=False)
lr_decay = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-10, mode='min', verbose=1) 
model.fit(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid), callbacks=[early_stop, lr_decay])

model.evaluate(x_test, y_test)

54
Epoch 1/100
107/657 [===>..........................] - ETA: 4s - loss: 0.7284 - auroc: 0.5142 - aupr: 0.1505

KeyboardInterrupt: ignored